In [ ]:
pwd

# 1. Business problem

* INX Future Inc ,(referred as INX ) , is one of the leading data analytics and automation solutions provider with over 15 years of global business presence. In recent years, the employee performance indexes are not healthy and this is becoming a growing concerns among the top management. There has been increased escalations on service delivery and client satisfaction levels came down by 8 percentage points.The CEO of company decided to analyse the current employee data and find the core underlying causes of this performance issues of the employees. Company also expects a clear indicators of non performing employees, so that any penalization of non-performing employee, if required, may not significantly affect other employee morals.



* ##### The following insights are expected from this project.


#### 1 Department-wise performance analysis

#### 2. Identify top 3 factors affecting performance

#### 3. Build a model to predict employee performance

#### 4. Give improvement recommendations



# 2. Set Up Directory Structure

In [ ]:
import os

folders = [
    "Project Summary/Requirement", "Project Summary/Analysis", "Project Summary/Summary",
    "data/raw", "data/processed",
    "src/Data Processing", "src/models", "src/visualization",
    "references"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

# 3. Importing Library

In [ ]:
#importiung the basic library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 4. Load the Dataset

In [ ]:
df = pd.read_excel(r'C:\Users\Dell\Music\DATAMITES\Projects\IABAC Project\Data.xls')
df

# 5. Basic Check

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
# Check that having object value
df.describe(include='O')

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.count()

In [ ]:
#getting catagorial and numerical Features
numerical_feature= [feature for feature in df.columns if df[feature].dtype!='O']
categorical_feature=[feature for feature in df.columns if df[feature].dtype=='O']
print('\nWe have {} numerical feature:{}'.format(len(numerical_feature),numerical_feature))
print('\n\nWe have {} categorical feature:{}'.format(len(categorical_feature),categorical_feature))

# 6. Domain Analysis

#### 1. JobSatisfaction, WorkLifeBalance, and Training are likely key performance influencers.

#### 2.Tenure and YearsInCurrentRole can reflect experience and adaptation.

#### 3. OverTime may increase output temporarily but may also reduce quality or satisfaction.

#### 4. Salary and JobRole may cause bias—so interpret their influence carefully.

#### 5. Avoid demographic bias: Age, Gender, and Marital Status should be handled cautiously or excluded from final models.

# 7. Exploratory Data Analysis (EDA)

## (a). Univariate Analysis

In [ ]:
## Univariant analysis of Numerical feature
plt.figure(figsize=(20,60))
for i in range(0,len(numerical_feature)):
    plt.subplot(10,2,i+1)
    sns.histplot(df[numerical_feature[i]])
    plt.xlabel(numerical_feature[i])
    plt.tight_layout()

### Insites

* Average age of employee in the INX Future Inc is 36 whereas the maximum employee lies in the age group of 25 to 45.
* Many employees resides near the office.
* Many employees has given rating as 3.0 for employee job environment.
* Many employees has worked for 0-1 company.
* most of the employee got salary hike of 0-12%.
* most of the employee has given performance rating as 3.

In [ ]:
# 2. Univariate analysis of categorical features
plt.figure(figsize=(20, 5 * len(categorical_feature))) 

for i, feature in enumerate(categorical_feature):
    plt.subplot(len(categorical_feature), 1, i + 1)
    sns.countplot(data=df, x=feature, palette='Set2')
    plt.xlabel(feature)
    plt.title(f'Distribution of {feature}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Insites

* Focus performance improvement or resource allocation where workforce is concentrated.
* Target training or hiring strategies based on dominant education backgrounds.
* Consider policies to promote diversity and equal opportunity.
* A high attrition count is a red flag. Investigate causes such as low job satisfaction or poor work-life balance.
* Consistent overtime could lead to burnout and declining performance. HR should monitor and regulate workloads.

## (b). Bivariate Analysis

In [ ]:
# 1. Biavariate analysis of Categorical vs Target (Boxplot)
# EmpDepartment vs PerformanceRating
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='EmpDepartment', y='PerformanceRating', palette='Set2')
plt.title("Performance Rating by Department")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 2. Biavariate analysis of Categorical vs Target (Barplot: Average Performance)
# OverTime vs PerformanceRating
plt.figure(figsize=(6, 4))
sns.boxplot(data=df, x='OverTime', y='PerformanceRating', palette='coolwarm')
plt.title("Performance Rating by Overtime")
plt.show()

In [ ]:
# 3. Biavariate analysis of Numerical vs Target (Scatterplot)
# ExperienceYearsAtThisCompany vs PerformanceRating
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, x='ExperienceYearsAtThisCompany', y='PerformanceRating', hue='EmpDepartment')
plt.title("Performance vs Tenure (by Department)")
plt.show()


In [ ]:
# 3. Numerical vs Target (Scatterplot)
# ExperienceYearsAtThisCompany vs PerformanceRating
pd.crosstab(df['OverTime'], df['Attrition'], normalize='index').plot(kind='bar', stacked=True)
plt.title("Attrition Rate by Overtime")
plt.ylabel("Proportion")
plt.show()

### Insites 
* EmpDepartment vs Performance - Some departments (e.g., Sales or HR) may have consistently lower ratings.
* OverTime vs Performance - Employees working overtime tend to have slightly lower performance.
* ExperienceYearsAtThisCompany vs Performance - Longer tenure often correlates with better performance.
* Attrition vs OverTime - High overtime is associated with higher attrition risk.

## (C). Multivariate Analysis

In [ ]:
sns.pairplot(df[['PerformanceRating', 'ExperienceYearsAtThisCompany', 'Age', 'EmpJobSatisfaction']])
plt.suptitle("Multivariate Relationship Overview", y=1.02)
plt.show()

### Insites 

* As expected, older employees tend to have more work experience. Including both in the model might introduce redundancy.
* These features overlap heavily in what they measure — employee tenure and role stability.
* Higher job level often means more experience, better salary hikes, and higher satisfaction — leading to better performance
* Training frequency alone doesn’t strongly relate to performance or experience.
* Older employees might live farther, but this doesn’t strongly affect job performance.

## Department-wise Performance Analysis

In [ ]:
###  1. Groupby Department and Calculate Mean Performance
dept_perf = df.groupby('EmpDepartment')['PerformanceRating'].mean().sort_values(ascending=False)
print(dept_perf)

In [ ]:
#visualisation
plt.figure(figsize=(10,5))
sns.barplot(x=dept_perf.index, y=dept_perf.values, palette='Set2')
plt.title('Average Performance Rating by EmpDepartment')
plt.xlabel('EmpDepartment')
plt.ylabel('Average Performance Rating')
plt.xticks(rotation=45)
plt.show()

In [ ]:
## 2. Attrition Rate by EmpDepartment
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})  # Convert to numeric
df.groupby('EmpDepartment')['Attrition'].mean().sort_values(ascending=False)

In [ ]:
# visualisation 
# Ensure Attrition is numeric
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

plt.figure(figsize=(10, 5))
attrition = df.groupby('EmpDepartment')['Attrition'].mean().sort_values()
sns.barplot(x=attrition.index, y=attrition.values, palette='Reds_r')
plt.title("Attrition Rate by EmpDepartment")
plt.ylabel("Attrition Rate")
plt.xticks(rotation=45)
plt.show()

In [ ]:
## 3. Average Job Satisfaction by EmpDepartment
df.groupby('EmpDepartment')['EmpJobSatisfaction'].mean().sort_values(ascending=False)

In [ ]:
# Visualisation 
plt.figure(figsize=(10, 5))
satisfaction = df.groupby('EmpDepartment')['EmpJobSatisfaction'].mean().sort_values()
sns.barplot(x=satisfaction.index, y=satisfaction.values, palette='Greens_r')
plt.title("Average Job Satisfaction by EmpDepartment")
plt.ylabel("Satisfaction Score")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# 4. OverTime Distribution by EmpDepartment
overtime_dist = pd.crosstab(df['EmpDepartment'], df['OverTime'], normalize='index')
overtime_dist.plot(kind='bar', stacked=True, figsize=(10,5), colormap='coolwarm')
plt.title('OverTime Proportion by EmpDepartment')
plt.ylabel('Proportion')
plt.show()


In [ ]:
# 5. Average Years in Current Role by Department
plt.figure(figsize=(10, 5))
years_role = df.groupby('EmpDepartment')['ExperienceYearsInCurrentRole'].mean().sort_values()
sns.barplot(x=years_role.index, y=years_role.values, palette='Purples_r')
plt.title("Average Years in Current Role by EmpDepartment")
plt.ylabel("Years")
plt.xticks(rotation=45)
plt.show()

# 8. Data processing

In [ ]:
df.info()

In [ ]:
# 1. Check for Missing Value
df.isnull().sum()

In [ ]:
# Handeling the missing value of Attrition
df['Attrition'].unique()

In [ ]:
df.drop(columns=['Attrition'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# 2. Check for Duplicate

df.duplicated().sum()

In [ ]:
df.dtypes


In [ ]:
numerical_cols = df.select_dtypes(include='number').columns.tolist()
print(numerical_cols)

In [ ]:
#checking outliers
plt.figure(figsize=(10,40))
plotnumber=1
for column in numerical_feature:
    if plotnumber<=28:
        ax=plt.subplot(18,2,plotnumber)
        sns.boxplot(df[column])
        plt.xlabel(column,fontsize=15)
    plotnumber+=1
plt.tight_layout()

### Outlier Handling
* If it is normal distribution we will use impirical rule or 3 sigma rule.
* if it is non normal we will use IQR for removing outliers.
* For Normal distribution the skewness and kurtosis should be in the range of -1 to 1.

In [ ]:
# TotalWorkExperienceInYears is normally distributed hence we use 3 sigma rule to remove outliers
lower_limit=df.TotalWorkExperienceInYears.mean() - 3*df.TotalWorkExperienceInYears.std()##calculating lower limit
print(lower_limit)

upper_limit=df.TotalWorkExperienceInYears.mean() + 3*df.TotalWorkExperienceInYears.std()#calculating upper limit
print(upper_limit)
df.loc[df['TotalWorkExperienceInYears']<lower_limit]#checking values which are less than minimum limit
df.loc[df['TotalWorkExperienceInYears']>upper_limit]#checking values which are greater than maximum limit
display(len(df.loc[df['TotalWorkExperienceInYears']>upper_limit])/1200) #checking percent of outliers
df.loc[df['TotalWorkExperienceInYears']>upper_limit,'TotalWorkExperienceInYears']=np.mean(df.TotalWorkExperienceInYears) #imputing value with mean as outliers percentage are less than 5%
display(df.loc[df['TotalWorkExperienceInYears']>upper_limit]) #reckeck

In [ ]:
#ExperienceYearsAtThisCompany is not normally distributed hence we use IQR rule to remove outliers
from scipy import stats
IQR = stats.iqr(df.ExperienceYearsAtThisCompany, interpolation = 'midpoint') #calculating Inter quantile range
display(IQR)
Q1=df.ExperienceYearsAtThisCompany.quantile(0.25)#defining 25% of data
Q3=df.ExperienceYearsAtThisCompany.quantile(0.75)##defining 75% of data
min_limit=Q1 - 1.5*IQR #setting minimum limit
max_limit=Q3 + 1.5*IQR #setting maximum limit
display(df.loc[df['ExperienceYearsAtThisCompany']<min_limit]) #checking values which are less than minimum limit
display(df.loc[df['ExperienceYearsAtThisCompany']>max_limit])#checking values which are greater than maximum limit
display(len(df.loc[df['ExperienceYearsAtThisCompany']>max_limit])/18278)
#imputing outlier with with median
df.loc[df['ExperienceYearsAtThisCompany']>max_limit,'ExperienceYearsAtThisCompany']=np.median(df.ExperienceYearsAtThisCompany)
df.loc[df['ExperienceYearsAtThisCompany']>max_limit] #checking whether outlier remove or not

In [ ]:
# ExperienceYearsInCurrentRole is normally distributed hence we use 3 sigma rule to remove outliers
lower_limit=df.ExperienceYearsInCurrentRole.mean() - 3*df.ExperienceYearsInCurrentRole.std()##calculating lower limit
display(lower_limit)

upper_limit=df.ExperienceYearsInCurrentRole.mean() + 3*df.ExperienceYearsInCurrentRole.std()#calculating upper limit
display(upper_limit)
display(df.loc[df['ExperienceYearsInCurrentRole']<lower_limit])#checking values which are less than minimum limit
display(df.loc[df['ExperienceYearsInCurrentRole']>upper_limit])#checking values which are greater than maximum limit
display(len(df.loc[df['ExperienceYearsInCurrentRole']>upper_limit])/1200) #checking percent of outliers
df.loc[df['ExperienceYearsInCurrentRole']>upper_limit,'ExperienceYearsInCurrentRole']=np.mean(df.ExperienceYearsInCurrentRole) #imputing value with mean as outliers percentage are less than 5%
display(df.loc[df['ExperienceYearsInCurrentRole']>upper_limit]) #reckeck

In [ ]:
# YearsSinceLastPromotion is not normally distributed hence we use IQR rule to remove outliers
IQR = stats.iqr(df.YearsSinceLastPromotion, interpolation = 'midpoint') #calculating Inter quantile range
display(IQR)
Q1=df.YearsSinceLastPromotion.quantile(0.25)#defining 25% of data
Q3=df.YearsSinceLastPromotion.quantile(0.75)##defining 75% of data
min_limit=Q1 - 1.5*IQR #setting minimum limit
max_limit=Q3 + 1.5*IQR #setting maximum limit
display(df.loc[df['YearsSinceLastPromotion']<min_limit]) #checking values which are less than minimum limit
display(df.loc[df['YearsSinceLastPromotion']>max_limit])#checking values which are greater than maximum limit
display(len(df.loc[df['YearsSinceLastPromotion']>max_limit])/18278)
#imputing outlier with with median
df.loc[df['YearsSinceLastPromotion']>max_limit,'YearsSinceLastPromotion']=np.median(df.YearsSinceLastPromotion)
df.loc[df['YearsSinceLastPromotion']>max_limit] #checking whether outlier remove or not

In [ ]:
# YearsWithCurrManager is normally distributed hence we use 3 sigma rule to remove outliers
lower_limit=df.YearsWithCurrManager.mean() - 3*df.YearsWithCurrManager.std()##calculating lower limit
print(lower_limit)

upper_limit=df.YearsWithCurrManager.mean() + 3*df.YearsWithCurrManager.std()#calculating upper limit
print(upper_limit)
display(df.loc[df['YearsWithCurrManager']<lower_limit])#checking values which are less than minimum limit
display(df.loc[df['YearsWithCurrManager']>upper_limit])#checking values which are greater than maximum limit
display(len(df.loc[df['YearsWithCurrManager']>upper_limit])/1200) #checking percent of outliers
df.loc[df['YearsWithCurrManager']>upper_limit,'YearsWithCurrManager']=np.mean(df.YearsWithCurrManager) #imputing value with mean as outliers percentage are less than 5%
display(df.loc[df['YearsWithCurrManager']>upper_limit]) #reckeck

In [ ]:
df.head()